# Исследование зависимости обучения сети от margin параметра Contrastive Loss

Обучение сиамских сетей на наборе MOT20_ext (2 пары). Используемая функция потерь - Contrastive Loss. Будет произведено обучение нескольких моделей на разных margin и выявлена зависимость обучения от margin

_другие гиперпараметры были подобраны ранее_

In [1]:
import albumentations as A
import torch
from torch.optim import Adam

from src.config import MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN
from src.models import SiameseBasicCNN
from src.models.reidentification import SiameseBasicCNN
from src.train import ContrastiveLoss, get_config, get_loaders, train_siamese
from src.train.loss import ContrastiveLoss
from src.transforms import get_norm_transform, get_resize_transform

## Загрузка датасета

In [2]:
dataset_config = get_config('mot20_ext-v1c')
dataset_config

{'dataset_config': 'mot20_ext-v1c',
 'dataset': 'mot20_ext',
 'dataset_use': 0.01,
 'test_proportion': 0.15,
 'val_proportion': 0.2,
 'test_proportion.1': nan,
 'batch_size': 16,
 'extra_parameters': {'visibility_threshold': 1.0,
  'negative_proportion': 0.5,
  'frame_distance': 0}}

In [3]:
resize_transform = get_resize_transform((MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN)) 
norm_transform = get_norm_transform()
transform = A.Compose([resize_transform, norm_transform])

In [4]:
train_loader, val_loader, test_loader = get_loaders(dataset_config, transform=transform)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Определить примерные границы margin

### Basic CNN, margin=0.001

In [6]:
model = SiameseBasicCNN().to(device)
lr = 1e-2
optimizer = Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
criterion = ContrastiveLoss(margin=0.001)

model, results = train_siamese(
    config=dataset_config,
    criterion=criterion,
    device=device,
    epoch_count=6,
    lr=lr,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    threshold=0.25,
    train_loader=train_loader,
    val_loader=val_loader,
)

Epoch 0/5:


/home/nick/reidentification/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 110/110 [00:17<00:00,  6.30it/s]

train Loss: 0.0292 Acc: 0.7415



100%|██████████| 33/33 [00:01<00:00, 20.98it/s]

val Loss: 0.0046 Acc: 0.7652


...

train Loss: 0.0004 Acc: 0.7347



100%|██████████| 33/33 [00:01<00:00, 22.05it/s]

val Loss: 0.0000 Acc: 0.8295
Model saved at basic_cnn.pth
Epoch 2/5:



 30%|███       | 33/110 [00:04<00:11,  6.77it/s]


KeyboardInterrupt: 

### Basic CNN, margin=0.01

In [7]:
model = SiameseBasicCNN().to(device)
lr = 1e-2
optimizer = Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
criterion = ContrastiveLoss(margin=0.01)

model, results = train_siamese(
    config=dataset_config,
    criterion=criterion,
    device=device,
    epoch_count=6,
    lr=lr,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    threshold=0.25,
    train_loader=train_loader,
    val_loader=val_loader,
)

Epoch 0/5:


100%|██████████| 110/110 [00:16<00:00,  6.82it/s]

train Loss: 0.0313 Acc: 0.7807



100%|██████████| 33/33 [00:01<00:00, 22.50it/s]

val Loss: 0.0001 Acc: 0.8712
Model saved at basic_cnn.pth
Epoch 1/5:


...

train Loss: 0.0000 Acc: 0.7102



100%|██████████| 33/33 [00:01<00:00, 22.62it/s]

val Loss: 0.0000 Acc: 0.8352
Epoch 2/5:



  9%|▉         | 10/110 [00:01<00:16,  6.18it/s]


KeyboardInterrupt: 

### Basic CNN, margin=0.1

In [8]:
model = SiameseBasicCNN().to(device)
lr = 1e-2
optimizer = Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
criterion = ContrastiveLoss(margin=0.1)

model, results = train_siamese(
    config=dataset_config,
    criterion=criterion,
    device=device,
    epoch_count=6,
    lr=lr,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    threshold=0.25,
    train_loader=train_loader,
    val_loader=val_loader,
)

Epoch 0/5:


100%|██████████| 110/110 [00:16<00:00,  6.76it/s]

train Loss: 0.0475 Acc: 0.7466



100%|██████████| 33/33 [00:01<00:00, 22.29it/s]

val Loss: 0.0050 Acc: 0.8485
Model saved at basic_cnn.pth
Epoch 1/5:


...

train Loss: 0.0034 Acc: 0.8136



100%|██████████| 33/33 [00:01<00:00, 22.48it/s]

val Loss: 0.0044 Acc: 0.8580
Epoch 3/5:



 15%|█▌        | 17/110 [00:02<00:14,  6.46it/s]


KeyboardInterrupt: 

### Basic CNN, margin=0.5

In [9]:
model = SiameseBasicCNN().to(device)
lr = 1e-2
optimizer = Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
criterion = ContrastiveLoss(margin=0.5)

model, results = train_siamese(
    config=dataset_config,
    criterion=criterion,
    device=device,
    epoch_count=6,
    lr=lr,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    threshold=0.25,
    train_loader=train_loader,
    val_loader=val_loader,
)

Epoch 0/5:


100%|██████████| 110/110 [00:16<00:00,  6.85it/s]

train Loss: 0.1223 Acc: 0.7727



100%|██████████| 33/33 [00:01<00:00, 22.12it/s]

val Loss: 0.1037 Acc: 0.8580
Model saved at basic_cnn.pth
Epoch 1/5:


...

val Loss: 0.0946 Acc: 0.9186
Epoch 5/5:



100%|██████████| 110/110 [00:16<00:00,  6.84it/s]

train Loss: 0.0847 Acc: 0.8284



100%|██████████| 33/33 [00:01<00:00, 22.01it/s]

val Loss: 0.0960 Acc: 0.9034


## Подбор параметра

In [10]:
for m in range(1, 6):
    model = SiameseBasicCNN().to(device)
    lr = 1e-2
    optimizer = Adam(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
    criterion = ContrastiveLoss(margin=m)

    model, results = train_siamese(
        config=dataset_config,
        criterion=criterion,
        device=device,
        epoch_count=6,
        lr=lr,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        threshold=0.25,
        train_loader=train_loader,
        val_loader=val_loader,
    )

Epoch 0/5:


/home/nick/reidentification/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 110/110 [00:15<00:00,  7.09it/s]

train Loss: 0.3910 Acc: 0.8205



100%|██████████| 33/33 [00:01<00:00, 20.99it/s]

val Loss: 0.3937 Acc: 0.8920
Model saved at basic_cnn.pth
Epoch 1/5:


...

val Loss: 9.5329 Acc: 0.9205
Epoch 5/5:



100%|██████████| 110/110 [00:16<00:00,  6.60it/s]

train Loss: 8.1757 Acc: 0.7568



100%|██████████| 33/33 [00:01<00:00, 19.41it/s]

val Loss: 9.4695 Acc: 0.9318
